In [1]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


### 데이터 로드

In [2]:
# 판다스로 훈련셋과 테스트셋 데이터 로드
train = pd.read_csv("finance_train.csv", encoding='utf-8')
test = pd.read_csv("finance_test.csv", encoding='utf-8')

print(train.shape)
print(test.shape)

(3412, 2)
(1434, 2)


In [3]:
# 훈련셋의 앞부분 출력
train.head(10)

,document,label
0,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로...",2
1,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ...",1
2,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...,0
3,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4...",0
4,ASPOCOMP의 성장기에 대한 자금 조달은 기술적으로 더 까다로운 HDI 인쇄 회...,0
5,2010년 4분기 Componenta의 순매출은 전년 동기의 7600만 유로에서 2...,0
6,영업이익은 2007년 해당 기간의 8.7 mn에서 13.1 mn으로 증가하여 순매출...,0
7,영업이익은 총 21.1 유로로 2007년 18.6 mn에서 증가하여 순매출의 9.7...,0
8,텔리아소네라 TLSN은 이번 제안이 핵심 사업 보유에 대한 지분을 늘리기 위한 전략...,0
9,"STORA ENSO, NORSKE SKOG, M-REAL, UPM-KYMMENE C...",0


In [4]:
train['label'].value_counts()

2    2008
0     958
1     446
Name: label, dtype: int64

### 전처리 - 훈련 데이터셋

In [5]:
# 리뷰 문장 추출
sentences = train['document']
sentences[:10]

0    Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로...
1    국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ...
2    새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
3    2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4...
4    ASPOCOMP의 성장기에 대한 자금 조달은 기술적으로 더 까다로운 HDI 인쇄 회...
5    2010년 4분기 Componenta의 순매출은 전년 동기의 7600만 유로에서 2...
6    영업이익은 2007년 해당 기간의 8.7 mn에서 13.1 mn으로 증가하여 순매출...
7    영업이익은 총 21.1 유로로 2007년 18.6 mn에서 증가하여 순매출의 9.7...
8    텔리아소네라 TLSN은 이번 제안이 핵심 사업 보유에 대한 지분을 늘리기 위한 전략...
9    STORA ENSO, NORSKE SKOG, M-REAL, UPM-KYMMENE C...
Name: document, dtype: object

In [6]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로 옮길 계획이 없다고 한다. [SEP]',
 '[CLS] 국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 해고와는 달리 회사는 사무직 직원 수를 줄였다고 일간 포스티메스가 보도했다. [SEP]',
 '[CLS] 새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가시키고 원자재 사용을 개선하여 생산 수익성을 높일 것이다. [SEP]',
 '[CLS] 2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 40% 범위의 장기적인 순매출 성장을 목표로 하고 있으며, 영업이익률은 순매출액의 10% - 20%를 목표로 하고 있습니다. [SEP]',
 '[CLS] ASPOCOMP의 성장기에 대한 자금 조달은 기술적으로 더 까다로운 HDI 인쇄 회로 기판 PCB에 점점 더 초점을 맞추면서 성장 전략을 공격적으로 추진하고 있다. [SEP]',
 '[CLS] 2010년 4분기 Componenta의 순매출은 전년 동기의 7600만 유로에서 2배인 13100만 유로로 증가했으며, 세전 손실 7백만 유로에서 세전 수익 0으로 전환되었다. [SEP]',
 '[CLS] 영업이익은 2007년 해당 기간의 8.7 mn에서 13.1 mn으로 증가하여 순매출액의 7.7%를 차지했다. [SEP]',
 '[CLS] 영업이익은 총 21.1 유로로 2007년 18.6 mn에서 증가하여 순매출의 9.7%를 차지했다. [SEP]',
 '[CLS] 텔리아소네라 TLSN은 이번 제안이 핵심 사업 보유에 대한 지분을 늘리기 위한 전략과 부합하며 에스티텔레콤의 고객 제안을 강화할 것이라고 밝혔다. [SEP]',
 '[CLS] STORA ENSO, NORSKE SKOG, M-REAL, UPM-KYMMENE Credit Suisse First Boston(CFSB)은 북유럽 삼림 그룹 중 4곳의 주식에 대한 공정가치를 상향 조정했

![대체 텍스트](https://mino-park7.github.io/images/2019/02/bert-input-representation.png)

BERT의 입력은 위의 그림과 같은 형식으로, Classification을 뜻하는 [CLS] 심볼이 제일 앞에 삽입됨. 파인튜닝시 출력에서 이 위치의 값을 사용하여 분류 <br> [SEP]은 Seperation을 가리키는데, 두 문장을 구분하는 역할을 함. 이 예제에서는 문장이 하나이므로 [SEP]도 하나만 삽입

In [7]:
# 라벨 추출
labels = train['label'].values
labels

array([2, 1, 0, ..., 1, 2, 1])

In [8]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로 옮길 계획이 없다고 한다. [SEP]
['[CLS]', 'Gran', '##에', '따르면', ',', '그', '회', '##사는', '회', '##사가', '성', '##장', '##하고', '있는', '곳', '##이지', '##만', ',', '모든', '생', '##산', '##을', '러시아', '##로', '옮', '##길', '계', '##획', '##이', '없다', '##고', '한다', '.', '[SEP]']


BERT는 형태소분석으로 토큰을 분리하지 않고, WordPiece라는 통계적인 방식을 사용 <br> 한 단어내에서 자주 나오는 글자들을 붙여서 하나의 토큰으로 만듦. 이렇게 하면 언어에 상관없이 토큰을 생성할 수 있다는 장점이 있음 (신조어 같이 사전에 없는 단어를 처리하기도 좋음) <br>
위의 결과에서 ## 기호는 앞 토큰과 이어진다는 의미. 토크나이저는 여러 언어의 데이터를 기반으로 만든 'bert-base-multilingual-cased'를 사용.

In [9]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,  13272,  10530,  59355,    117,   8924,   9998,  64379,
         9998,  49636,   9434,  13890,  12453,  13767,   8895,  44359,
        19105,    117,  25701,   9420,  21386,  10622,  62834,  11261,
         9584, 118666,   8887, 103155,  10739,  39218,  11664,  16139,
          119,    102,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

보통 딥러닝 모델에는 토큰 자체를 입력으로 넣을 수 없음 -> 임베딩 레이어에서 토큰을 숫자로 된 인덱스로 변환하여 사용. BERT의 토크나이저는 {단어토큰:인덱스}로 구성된 단어 사전을 갖고 있는데, 이를 참조하여 토큰을 인덱스로 바꿈

In [10]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [11]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([  101,  9638, 12945, 14863, 11018, 19789,  9323, 25549, 16323, 21155,
        21928,   121,   119, 10197, 42815, 11261, 10459,  9330, 21928, 40032,
        10622,  9706, 37568, 14843, 21371,  9672, 34951, 12609,   119,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [12]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

### 전처리 - 테스트 데이터셋

In [13]:
# 리뷰 문장 추출
sentences = test['document']
sentences[:10]

0    테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
1    2010년 3분기 순매출은 5.2%, 영업이익은 34.9% 증가한 23.5MN을 기...
2    핀란드 하미나 터미널에서 인도하는 휘발유 7,200t에 대한 구매협약이 올 9월 플...
3    연결 순매출액은 16% 증가하여 74.8백만 유로에 달했고, 영업이익은 전년도의 0...
4    주조 공장 부문은 매출이 2006년 해당 기간의 57.5 mn에서 63.1 mn으로...
5    헬싱키 (AFX) &shy; 노키아가 산요와 손잡고 3G 단말기를 제조할 계획을 발...
6    핀란드 인셉트 코퍼레이션의 자회사인 인셉트 컨트랙트 제조 서비스 Pvt Ltd는 2...
7    이사회는 2009년 주당 0.08유로에서 2010년 주당 0.12유로의 배당금을 제...
8    생명보험 순이익은 3.2mn에서 6.8mn으로 2배 증가했고, 손해보험 순이익은 2...
9    2007년 같은 기간에 비해 9개월 영업이익은 .1억 유로에서 증가하였고 순매출은 ...
Name: document, dtype: object

In [14]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평방미터 이상의 면적을 단계적으로 개발할 계획이라고 성명은 밝혔다. [SEP]',
 '[CLS] 2010년 3분기 순매출은 5.2%, 영업이익은 34.9% 증가한 23.5MN을 기록했다. [SEP]',
 '[CLS] 핀란드 하미나 터미널에서 인도하는 휘발유 7,200t에 대한 구매협약이 올 9월 플랫츠 평균지수에서 네스테오일 OYJ와 월 8달러(약 8만 원)로 체결됐다. [SEP]',
 '[CLS] 연결 순매출액은 16% 증가하여 74.8백만 유로에 달했고, 영업이익은 전년도의 0.7백만 유로에 비해 0.9백만 유로에 달했다. [SEP]',
 '[CLS] 주조 공장 부문은 매출이 2006년 해당 기간의 57.5 mn에서 63.1 mn으로 9.7% 증가했으며, 기계 공장 부문 매출은 2006년 해당 기간의 35.4 mn에서 41.2 mn으로 16.4% 증가했다고 보고했습니다. [SEP]',
 '[CLS] 헬싱키 (AFX) &shy; 노키아가 산요와 손잡고 3G 단말기를 제조할 계획을 발표한 후, 노키안 타이어가 4분기 수익보고서를 발표한 후, 주가는 분석가들의 예상을 깨고 상승 마감했다고 딜러들은 말했다. [SEP]',
 '[CLS] 핀란드 인셉트 코퍼레이션의 자회사인 인셉트 컨트랙트 제조 서비스 Pvt Ltd는 2007-2008년까지 수익을 두 배로 늘릴 계획이다. [SEP]',
 '[CLS] 이사회는 2009년 주당 0.08유로에서 2010년 주당 0.12유로의 배당금을 제안할 예정이다. [SEP]',
 '[CLS] 생명보험 순이익은 3.2mn에서 6.8mn으로 2배 증가했고, 손해보험 순이익은 2009년 해당 기간 1.5mn에서 5.2mn으로 증가했다. [SEP]',
 '[CLS] 2007년 같은 기간에 비해 9개월 영업이익은 .1억 유로에서 증가하였고 순매출은 .5억 유로에서 증가하였다. [SEP]']

In [15]:
# 라벨 추출
labels = test['label'].values
labels

array([2, 0, 0, ..., 1, 1, 1])

In [16]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평방미터 이상의 면적을 단계적으로 개발할 계획이라고 성명은 밝혔다. [SEP]
['[CLS]', '테', '##크', '##노', '##폴', '##리스', '##는', '컴', '##퓨', '##터', '기', '##술', '##과', '통', '##신', '분', '##야', '##에서', '일', '##하는', '회', '##사', '##들을', '유', '##치', '##하기', '위해', '10', '##만', '평', '##방', '##미', '##터', '이상의', '면', '##적을', '단', '##계', '##적으로', '개발', '##할', '계', '##획', '##이', '##라고', '성', '##명은', '밝혔다', '.', '[SEP]']


In [17]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9866,  20308,  28981, 119402,  79940,  11018,   9802,
       119410,  21876,   8932,  51945,  11882,   9879,  25387,   9367,
        21711,  11489,   9641,  12178,   9998,  12945,  25820,   9625,
        18622,  22440,  19905,  10150,  19105,   9926,  42337,  22458,
        21876,  80607,   9279,  80416,   9059,  21611,  17022, 110176,
        14843,   8887, 103155,  10739,  59894,   9434,  78311,  99896,
          119,    102,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [18]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [19]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   9866,  20308,  28981, 119402,  79940,  11018,   9802, 119410,
         21876,   8932,  51945,  11882,   9879,  25387,   9367,  21711,  11489,
          9641,  12178,   9998,  12945,  25820,   9625,  18622,  22440,  19905,
         10150,  19105,   9926,  42337,  22458,  21876,  80607,   9279,  80416,
          9059,  21611,  17022, 110176,  14843,   8887, 103155,  10739,  59894,
          9434,  78311,  99896,    119,    102,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [20]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

<br>
<br>

# **모델 생성**

In [21]:
# GPU 디바이스 이름
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 확인
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


2022-06-03 08:43:35.564208: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-03 08:43:37.217565: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:0 with 9643 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:18:00.0, compute capability: 7.5
2022-06-03 08:43:37.218299: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:1 with 9643 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:86:00.0, compute capability: 7.5
2022-06-03 08:43:37.218874: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:2 with 9643 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti

In [22]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 3 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti


In [23]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)  # 긍정, 부정, 중립
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

![대체 텍스트](http://www.mccormickml.com/assets/BERT/padding_and_mask.png)

사전훈련된 BERT는 다양한 문제로 전이학습이 가능함. 여기서는 위의 그림과 같이 한 문장을 분류. 뉴스 기사 한 문장이 입력으로 들어가면, 긍정/부정/중립으로 구분 -> 모델의 출력에서 [CLS] 위치인 첫 번째 토큰에 새로운 레이어를 붙여서 파인튜닝 (Huggning Face는 BertForSequenceClassification() 함수를 제공하기 때문에 쉽게 구현이 가능)

In [24]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 10

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/home/seominseok/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### 모델 학습

In [25]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [26]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [27]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 출력 로짓 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.72
  Training epcoh took: 0:00:25

Running Validation...
  Accuracy: 0.78
  Validation took: 0:00:01

======== Epoch 2 / 10 ========
Training...

  Average training loss: 0.46
  Training epcoh took: 0:00:25

Running Validation...
  Accuracy: 0.77
  Validation took: 0:00:01

======== Epoch 3 / 10 ========
Training...

  Average training loss: 0.33
  Training epcoh took: 0:00:26

Running Validation...
  Accuracy: 0.79
  Validation took: 0:00:01

======== Epoch 4 / 10 ========
Training...

  Average training loss: 0.24
  Training epcoh took: 0:00:26

Running Validation...
  Accuracy: 0.75
  Validation took: 0:00:01

======== Epoch 5 / 10 ========
Training...

  Average training loss: 0.16
  Training epcoh took: 0:00:26

Running Validation...
  Accuracy: 0.76
  Validation took: 0:00:01

======== Epoch 6 / 10 ========
Training...

  Average training loss: 0.12
  Training epcoh took: 0:00:26

Running Validation...
  Accu

### 테스트 데이터셋 평가

In [28]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 출력 로짓 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.79
Test took: 0:00:04


### 새로운 문장 테스트

In [29]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [30]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 출력 로짓 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [31]:
logits = test_sentences(['계속해서 새로운 포맷이 나오고 있고 우리는 그 새로운 포맷에 맞춰 새로운 광고 상품과 솔루션을 만들어야 한다"고 했다.'])

print(logits)
print(np.argmax(logits))  # 0 - 긍정, 1 - 부정, 2- 중립

[[-0.5925627 -3.7500825  3.8602684]]
2


### 모델 저장

In [32]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.txt',
 './model_save/added_tokens.json')

### 모델 로드

In [33]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [34]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


2022-06-03 08:48:15.737318: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:0 with 9643 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:18:00.0, compute capability: 7.5
2022-06-03 08:48:15.737741: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:1 with 9643 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:86:00.0, compute capability: 7.5
2022-06-03 08:48:15.738112: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:2 with 9643 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:af:00.0, compute capability: 7.5


In [35]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 3 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti


In [36]:
output_dir = './model_save/'

# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [37]:
logits = test_sentences(['네이버의 매출과 이익을 주로 담당하는 이커머스, 서치플랫폼 분야와 네이버 웹툰 등 콘텐츠 분야는 지속적으로 성장하고 있다.'])

print(logits)
print(np.argmax(logits))  # 0 - 긍정, 1 - 부정, 2- 중립

[[ 3.1127222  -3.6534476  -0.05307636]]
0
